The following approach was original inspired by this [tutorial](https://www.youtube.com/watch?v=FB5EdxAGxQg&ab_channel=codebasics) - however it was later realized that due to the quadratic time complexity the [sklearn.svm.SVC](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) may not be right for our project, since it does not support Stochastic Gradient Descent using batches.

##**Loading the dataset**

In [ ]:
import torchvision.transforms as transforms
import torchvision
import torch
from random import seed
seed(40)
torch.manual_seed(40)

# Transformations applied to images as they are loaded by the Pytorch dataloader
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
)

#where we'll store the data
google_drive_path = './drive/MyDrive/Colab Notebooks/3337_Project'

#download our data 
trainset = torchvision.datasets.CIFAR10(root=google_drive_path, train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root=google_drive_path, train=False, download=True, transform=transform)
#the above code is really just used for DLing all data
#we still need to manually transform it (normalize it)
#and we still need to split it into training and testing data

#don't need these anymore
del trainset 
del testset

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./drive/MyDrive/Colab Notebooks/3337_Project/cifar-10-python.tar.gz to ./drive/MyDrive/Colab Notebooks/3337_Project
Files already downloaded and verified


##**Defining functions for loading the data**

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt

np.random.seed(40)

def unpickle(file): #adapted from https://www.cs.toronto.edu/~kriz/cifar.html
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def load_CIFAR_batch(filename):
  """ load single batch of cifar """
  datadict = unpickle(filename)
  X = datadict[b'data']
  Y = datadict[b'labels']
  X = X.reshape(10000, 3, 32, 32).transpose(0,2,3,1).astype("float")
  Y = np.array(Y)
  return X, Y

def load_CIFAR10(ROOT): #adapted from tutorial #this splits the data into training and testing sets
  """ load all of cifar """
  xs = []
  ys = []
  for b in range(1,6):
    f = ROOT + '/data_batch_%d'%(b,)
    X, Y = load_CIFAR_batch(f)
    xs.append(X)
    ys.append(Y)    
  Xtr = np.concatenate(xs)
  Ytr = np.concatenate(ys)
  del X, Y
  Xte, Yte = load_CIFAR_batch(ROOT + '/test_batch')
  return Xtr, Ytr, Xte, Yte

##**Loading the data and preprocessing**

In [ ]:
import pandas as pd
cifar10_dir = './drive/MyDrive/Colab Notebooks/3337_Project/cifar-10-batches-py' #changed

X_train_4D_unNorm, y_train, X_test_4D_unNorm, y_test = load_CIFAR10(cifar10_dir) #splits into training and testing sets

In [ ]:
data = [X_train_4D_unNorm, y_train, X_test_4D_unNorm, y_test]
for each in data:
  print(each.shape)
#still need to reshape and normalize

(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [ ]:
#reshape the data
X_train_2D_unNorm = np.reshape(X_train_4D_unNorm, (X_train_4D_unNorm.shape[0],-1))
X_test_2D_unNorm = np.reshape(X_test_4D_unNorm, (X_test_4D_unNorm.shape[0],-1))

#now we don't need the 4D data anymore
del X_train_4D_unNorm
del X_test_4D_unNorm

In [ ]:
#here we show that we successfully transformed the 4D set of images to a 2D set of images
#each column is named column,row,rgb
print(X_train_2D_unNorm.shape)
print(X_test_2D_unNorm.shape)

(50000, 3072)
(10000, 3072)


In [ ]:
#now let's produce our pd.DataFrames

columnLabels = [] #we'll feed this to our pd.DataFrame to label each column for each image
for i in range(32):
  for j in range(32):
    for color in ['r','g','b']:
      columnLabels.append(str(i)+','+str(j)+','+color)

X_train_unNorm = pd.DataFrame(X_train_2D_unNorm, columns=columnLabels)
X_test_unNorm = pd.DataFrame(X_test_2D_unNorm, columns=columnLabels)

#don't need X_train_2D_unNorm or X_test_2D_unNorm anymore
del X_train_2D_unNorm
del X_test_2D_unNorm

###**Normalization**
We are told in the [sklearn.linear_model.SGDClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html#sklearn.linear_model.SGDClassifier) that "For best results using the default learning rate schedule, the data should have zero mean and unit variance." so we will standardize using [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler).

In [ ]:
#let's standardize
from sklearn.preprocessing import StandardScaler
X_train = pd.DataFrame(StandardScaler().fit_transform(X_train_unNorm), columns=columnLabels)
#fit_transform uses one argument both to determine the mins and maxs (scaling params) and to do the transform
#to normalize our X_test, we want to use the mins and maxs from the train data, since it's such a bigger sample, so we'll do this normalization in steps, first we'll call fit on the training data, then we'll call transform on the test data
X_test = pd.DataFrame(StandardScaler().fit(X_train_unNorm).transform(X_test_unNorm), columns=columnLabels) 

##**Training**
Found this [discussion](https://stackoverflow.com/questions/24617356/sklearn-sgdclassifier-partial-fit) helpful. Note that the SGDClassifier parameter shuffle being set to True apparently does not actually produce shuffling. Shuffling was not implemented manually, but it could be with more time.




In [ ]:
#now we are ready to make our Support Vector Classifier (a type of support vector machine)
from sklearn.linear_model import SGDClassifier
import random

model = SGDClassifier()
# for i in range(10):
#   model.partial_fit(X_train[0:5000*(i+1)], y_train[0:5000*(i+1)], classes=np.unique(y_train))

miniBatches = 10
for i in range(miniBatches):
  startIndex = int(i*len(X_train.index)/miniBatches)
  endIndex = int((i+1)*len(X_train.index)/miniBatches)
  model.partial_fit(X_train[startIndex:endIndex], y_train[startIndex:endIndex], classes=np.unique(y_train))


In [ ]:
#here we save the model
import pickle
pickle_filename = "./drive/MyDrive/Colab Notebooks/3337_Project/linearSVM_trained_with_batches.pkl"
pickle.dump(model, open(pickle_filename,'wb'))

##**Scoring**


In [ ]:
model.score(X_test, y_test)

0.3715